<a href="https://colab.research.google.com/github/rrahul2203/NLP/blob/main/Document_based_question_anserwing_with_Langchain_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade langchain openai  -q
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
!apt-get install poppler-utils
!pip install tiktoken -q
!pip install pinecone-client -q

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXX"

In [ ]:
from langchain.document_loaders import DirectoryLoader

directory = '/content/data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

In [ ]:
print(docs[0].page_content)

In [ ]:
print(docs[1].page_content)

In [ ]:
import openai
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model_name="ada")

query_result = embeddings.embed_query("Hello world")
len(query_result)

In [ ]:
import pinecone
from langchain.vectorstores import Pinecone
# initialize pinecone
pinecone.init(
    api_key="f7aa89af-ac99-4619-8914-8d08740f7b38",  # find at app.pinecone.io
    environment="us-east-1-aws"  # next to api key in console
)

index_name = "langchain-demo"

index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [ ]:
def get_similiar_docs(query,k=2,score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query,k=k)
  else:
    similar_docs = index.similarity_search(query,k=k)
  return similar_docs

query = "How is india's economy"
similar_docs = get_similiar_docs(query)
similar_docs

In [ ]:
from langchain.llms import OpenAI

model_name = "gpt-4"
llm = OpenAI(model_name=model_name)

In [ ]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer =  chain.run(input_documents=similar_docs, question=query)
  return  answer

In [ ]:
query = "How is india's economy"
get_answer(query)

In [ ]:
query = "How relations between india and us has improved?"
get_answer(query)